In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil
import sys
import tqdm
import multiprocessing as mp
from config import *

path = result_parent_directory + os.sep + 'step4c_btms_sizing_sensitivity_wait_time'
path_no_wait = result_parent_directory + os.sep + 'step4_btms_sizing_sensitivity'

result_directory = path + os.sep + 'analysis'
os.makedirs(result_directory, exist_ok=True)
figure_directory = path + os.sep + 'figures'
os.makedirs(figure_directory, exist_ok=True)

sizing_results_path = os.path.join(path, 'sizing_results')

print('Loading results from {}'.format(sizing_results_path))
print('Loading no wait results from {}'.format(path_no_wait))
print('result_directory: {}'.format(result_directory))
print('figure_directory: {}'.format(figure_directory))

Loading results from results\advancedScenario7\step4c_btms_sizing_sensitivity_wait_time\sizing_results
Loading no wait results from results\advancedScenario7\step4_btms_sizing_sensitivity
result_directory: results\advancedScenario7\step4c_btms_sizing_sensitivity_wait_time\analysis
figure_directory: results\advancedScenario7\step4c_btms_sizing_sensitivity_wait_time\figures


In [2]:
# sort files into folders with a_cost_sizing and d_wait_cost as folder names
# a_cost_sizing is top folder, d_wait_cost is subfolder

folder_sorted_sizing_begin = 'sorted_sizing'
for folder1 in os.listdir(sizing_results_path):
    if not folder1.startswith(folder_sorted_sizing_begin):
        path_folder = os.path.join(sizing_results_path, folder1)
        # load a_cost_sizing and d_wait_cost from first file in dir
        file = os.listdir(path_folder)[0]
        path_file = os.path.join(path_folder, file)
        df = pd.read_csv(path_file)
        a = df['param: btms size, a,b_sys,b_cap,b_loan,c,d_wait_cost'][1] * (365/12)
        d = df['param: btms size, a,b_sys,b_cap,b_loan,c,d_wait_cost'][6]
        # create folder structure
        # round a 
        path_folder_new = os.path.join(sizing_results_path, folder_sorted_sizing_begin, 'a_' + str(round(a)))
        os.makedirs(path_folder_new, exist_ok=True)
        path_folder_new = os.path.join(path_folder_new, 'd_' + str(int(d)))
        os.makedirs(path_folder_new, exist_ok=True)
        # move files
        for file in os.listdir(path_folder):
            path_file = os.path.join(path_folder, file)
            shutil.move(path_file, path_folder_new)
        # remove empty folder
        os.rmdir(path_folder)

In [ ]:
from functions.stats_sizing_function import stats_sizing_function_wait_time
# calculate stats for each charging station and save them in dict: top-level: a_cost_sizing, second-level: d_wait_cost
stats_dict = {}

for folder1 in os.listdir(sizing_results_path):
    if folder1.startswith(folder_sorted_sizing_begin):
        path_folder1 = os.path.join(sizing_results_path, folder1)
        # read a value from folder name
        a_value = float(folder1.split('_')[1])
        for folder2 in os.listdir(path_folder1):
            # read d value from folder name
            d_value = float(folder2.split('_')[1])
            path_folder2 = os.path.join(path_folder1, folder2)
            stats_i = []
            for file in os.listdir(path_folder2):
                stats_i.append(stats_sizing_function_wait_time(pd.read_csv(os.path.join(path_folder2, file)), timestep))
            